In [3]:
import os
from uuid import uuid4
from pinecone import Pinecone

pinecone_api_key = os.environ.get("PINECONE_API_KEY", '912b76d8-5a64-4f46-97e3-099a8e448389')

pc = Pinecone(api_key=pinecone_api_key)

data = [
    "## Кога е најдобро да вежбам во текот на денот? Нема физиолошки придобивки или недостатоци при изборот на време за вежбање. Главната придобивка е ментална. НАЈДОБРОТО ВРЕМЕ за вежбање е она кое одговара на вас согласно вашите обврски, работа и активности. Нека биде време во кое е најмалку веројатно дека ќе правите изговори.",
    """
        ## Колку пати неделно да вежбам за да имам резултати? 
        Ако во моментот имате 0 активност и само 1 тренинг во неделата може да направи разлика. Поставување на нереални достигнувања и вежбање пример секој ден може да направи само контраефект. Запомнете дека секогаш можете да додадете плус тогаш кога ќе се чувствувате подготвено. Во секој случај на самиот почеток 2-3 пати неделно активност ќе биде доволно.
    """,
    """
## Колку време ќе ми треба да видам резултати? 
Ова е многу различно за секој човек, но генерално 3 до 4 месеци конзистентност се доволни да видите видилива разлика која и другите ќе можат да ја забележат. Сепак, освен физички, поставете си и емоционални и ментални цели кои ќе бидат со вас во процесот на менување на навиките и ќе ви помогнат полесно да чекорите кон целта на целосно подобра верзија од себе.
    """,
    """
## Кои тренинзи ќе ми дадат најубав ефект? 
Има многу различни начини и стилови на вежбање, но најголем ефект ќе имате од тренинзите во кои вие ќе уживате најмногу и кои ќе можете лесно да ги вклопите во дневен распоред. Ако сте мајка која работи и има многу малку слободно време за себе, можеби вежбањето дома кратки, функционални тренинзи ќе биде најдобро. Важно е да бидете активни, не заборавјте дека 1 е повеќе од 0.
    """,
    """
## Дали да јадам пред вежбање? 
Потрудете се последниот оброк пред вежбање да биде барем 2-3 часа пред тренингот. Не е убаво да вежбате со полн стомак. Ако вежбате наутро нема да имате проблем со тоа, но доколку вежбате попладне, потрудете се да не вежбате со полн стомак.
    """,
]

docs = [
    {
        "id": str(uuid4()),
        "text": doc,
    }
    for doc in data
]

embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d["text"] for d in docs],
    parameters={"input_type": "passage", "truncate": "END"},
)

print(embeddings)


EmbeddingsList(
  model='multilingual-e5-large',
  data=[
    {'values': [0.0098876953125, -0.0286102294921875, ..., -0.040740966796875, 0.006633758544921875]},
    {'values': [0.0079193115234375, -0.0240325927734375, ..., -0.0233917236328125, 0.01375579833984375]},
    {'values': [0.0024242401123046875, -0.0248565673828125, ..., -0.054290771484375, 0.00897216796875]},
    {'values': [0.0223236083984375, -0.0087890625, ..., -0.024749755859375, 0.01207733154296875]},
    {'values': [0.0226287841796875, -0.027130126953125, ..., -0.0322265625, 0.00424957275390625]}
  ],
  usage={'total_tokens': 459}
)


In [4]:
index_name = "p-docs"
index = pc.Index(index_name)

records = []
for d, e in zip(docs, embeddings):
    records.append({
        'id': d['id'],
        'values': e['values'],
        'metadata': {'text': d['text']}
    })

index.upsert(vectors=records, namespace='article1')


{'upserted_count': 5}

In [7]:

print(index.describe_index_stats())


{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'article1': {'vector_count': 5}},
 'total_vector_count': 5}


In [11]:
pc_model = "multilingual-e5-large"
pc_index_name = "p-docs"
def pinecone_answer(query):
    pc = Pinecone(api_key=pinecone_api_key)
    query_embedding = pc.inference.embed(
        model=pc_model, inputs=[query], parameters={"input_type": "query"}
    )
    index = pc.Index(pc_index_name)
    results = index.query(
        vector=query_embedding[0].values,
        namespace='article1',
        top_k=3,
        include_values=False,
        include_metadata=True,
    )

    return results


In [14]:
matches = pinecone_answer('Колку време до резултати')

In [ ]:
matches['matches'][0]['metadata']['text']


'\n## Колку време ќе ми треба да видам резултати? \nОва е многу различно за секој човек, но генерално 3 до 4 месеци конзистентност се доволни да видите видилива разлика која и другите ќе можат да ја забележат. Сепак, освен физички, поставете си и емоционални и ментални цели кои ќе бидат со вас во процесот на менување на навиките и ќе ви помогнат полесно да чекорите кон целта на целосно подобра верзија од себе.\n    '